## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
updated_applications = application_df.drop(columns=['NAME' , 'EIN'], axis=1)

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = updated_applications['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [18]:
# Look at APPLICATION_TYPE value counts for binning
reduced_applications = updated_applications['APPLICATION_TYPE'].value_counts()[updated_applications['APPLICATION_TYPE'].value_counts()>10]
reduced_applications

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
Name: APPLICATION_TYPE, dtype: int64

In [19]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = updated_applications['APPLICATION_TYPE'].value_counts(ascending=False).index[8:].tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [20]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df['CLASSIFICATION'].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
Other     4144
C3000     1918
Name: CLASSIFICATION, dtype: int64

In [21]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_count[classification_count>1]
classification_count

C1000    17326
C2000     6074
C1200     4837
Other     4144
C3000     1918
Name: CLASSIFICATION, dtype: int64

In [22]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts(ascending=False).index[4:].tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [23]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_data = pd.get_dummies(application_df, dtype=float)

In [24]:
# Split our preprocessed data into our features and target arrays
x = numeric_data.drop(columns = ['IS_SUCCESSFUL']).values
y = numeric_data['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, Y_train, Y_test = train_test_split(x,y, random_state=42)

In [25]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train[0])
print("Input Features =", number_input_features)
hidden_nodes_layer1 = 10
print("Hidden Nodes - Layer 1 =", hidden_nodes_layer1)
hidden_nodes_layer2 = 15
print("Hidden Nodes - Layer 2 =", hidden_nodes_layer2)
hidden_nodes_layer3 = 20
print("Hidden Nodes - Layer 3 =", hidden_nodes_layer3)
hidden_nodes_layer4 = 20
print("Hidden Nodes - Layer 4 =", hidden_nodes_layer4)

#Keras Sequential Model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Input Features = 19610
Hidden Nodes - Layer 1 = 10
Hidden Nodes - Layer 2 = 15
Hidden Nodes - Layer 3 = 20
Hidden Nodes - Layer 4 = 20
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 10)                196110    
                                                                 
 dense_12 (Dense)            (None, 15)                165       
                                                                 
 dense_13 (Dense)            (None, 20)                320       
                                                                 
 dense_14 (Dense)            (None, 20)                420       
                                                                 
 dense_15 (Dense)            (None, 1)                 21        
                                                                 
Total params: 197,036
Trainable params: 197,036
Non

In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled,Y_train,epochs=150, validation_split=0.15)

Epoch 1/150
684/684 [==============================] - 5s 6ms/step - loss: 0.5427 - accuracy: 0.7464 - val_loss: 0.5005 - val_accuracy: 0.7489
Epoch 2/150
684/684 [==============================] - 4s 5ms/step - loss: 0.3052 - accuracy: 0.8626 - val_loss: 0.5680 - val_accuracy: 0.7512
Epoch 3/150
684/684 [==============================] - 4s 5ms/step - loss: 0.1512 - accuracy: 0.9434 - val_loss: 0.6089 - val_accuracy: 0.7852
Epoch 4/150
684/684 [==============================] - 4s 5ms/step - loss: 0.1152 - accuracy: 0.9563 - val_loss: 0.6420 - val_accuracy: 0.7442
Epoch 5/150
684/684 [==============================] - 4s 5ms/step - loss: 0.0999 - accuracy: 0.9600 - val_loss: 0.6545 - val_accuracy: 0.7367
Epoch 6/150
684/684 [==============================] - 4s 5ms/step - loss: 0.0946 - accuracy: 0.9621 - val_loss: 0.9162 - val_accuracy: 0.6655
Epoch 7/150
684/684 [==============================] - 4s 5ms/step - loss: 0.0930 - accuracy: 0.9616 - val_loss: 1.2627 - val_accuracy: 0.6129

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,Y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 1.0426 - accuracy: 0.6223 - 550ms/epoch - 2ms/step
Loss: 1.042646884918213, Accuracy: 0.6222740411758423


In [31]:
# Export our model to HDF5 file
nn.save('../AlphabetSoupCharity_Optimization3.h5')